# Laboratorium 4 (4 pkt.)

Celem czwartego laboratorium jest zapoznanie się oraz zaimplementowanie algorytmów głębokiego uczenia aktywnego. Zaimplementowane algorytmy będą testowane z wykorzystaniem wcześniej przygotowanych środowisk: *FrozenLake* i *Pacman* oraz środowiska z OpenAI - *CartPole*.


Dołączenie standardowych bibliotek

In [1]:
from collections import deque
import gym
import numpy as np
from tqdm import tqdm

Dołączenie bibliotek ze środowiskami:

In [2]:
from env.FrozenLakeMDP import frozenLake
from env.FrozenLakeMDPExtended import frozenLake as frozenLakeExtended

Dołączenie bibliotek do obsługi sieci neuronowych

In [3]:
import tensorflow as tf

## Zadanie 1 - Deep Q-Network

<p style='text-align: justify;'>
Celem ćwiczenie jest zaimplementowanie algorytmu Deep Q-Network. Wartoscią oczekiwaną sieci jest:
\begin{equation}
        Q(s_t, a_t) = r_{t+1} + \gamma \text{max}_a Q(s_{t + 1}, a)
\end{equation}
</p>

In [4]:
class DQNAgent:
    def __init__(self, action_size, learning_rate, model, environment):
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = learning_rate
        self.model = model
        self.env = environment

    def remember(self, state, action, reward, next_state, done):
        #Function adds information to the memory about last action and its results
        self.memory.append((state, action, reward, next_state, done)) 

    def get_action(self, state):
        """
        Compute the action to take in the current state, including exploration.
        With probability self.epsilon, we should take a random action.
            otherwise - the best policy action (self.get_best_action).

        Note: To pick randomly from a list, use random.choice(list).
              To pick True or False with a given probablity, generate uniform number in [0, 1]
              and compare it with your probability
        """

        if np.random.random() < self.epsilon:
            return np.random.choice(self.env.get_possible_actions(state))

        return self.get_best_action(state)


  
    def get_best_action(self, state):
        """
        Compute the best action to take in a state.
        """

        prediction = self.model.predict(state, verbose=0)
        best_action = np.argmax(prediction[0])

        return best_action

    def replay(self, batch_size):
        """
        Function learn network using randomly selected actions from the memory. 
        First calculates Q value for the next state and choose action with the biggest value.
        Target value is calculated according to:
                Q(s,a) := (r + gamma * max_a(Q(s', a)))
        except the situation when the next action is the last action, in such case Q(s, a) := r.
        In order to change only those weights responsible for chosing given action, the rest values should be those
        returned by the network for state state.
        The network should be trained on batch_size samples.
        """

        sample_idx = np.random.choice(len(self.memory), size=batch_size, replace=False)
        batch_list = [self.memory[idx] for idx in sample_idx]

        for sample in batch_list:
            state, action, reward, next_state, done = sample

            target = reward + self.gamma * np.amax(self.model.predict(next_state, verbose=0)[0])

            prediction = self.model.predict(state, verbose=0)
            prediction[0][action] = target
            self.model.fit(state, prediction, epochs=1, verbose=0)

    def update_epsilon_value(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

Czas przygotować model sieci, która będzie się uczyła poruszania po środowisku *FrozenLake*, warstwa wejściowa powinna mieć tyle neuronów ile jest możlliwych stanów, warstwa wyjściowa tyle neuronów ile jest możliwych akcji do wykonania:

In [5]:
env = frozenLake("8x8")

state_size = env.get_number_of_states()
action_size = len(env.get_possible_actions(None))
learning_rate = 0.001

init = tf.keras.initializers.GlorotNormal
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(25, input_shape=(state_size,), activation='relu', kernel_initializer=init))
model.add(tf.keras.layers.Dense(25, activation='relu', kernel_initializer=init))
model.add(tf.keras.layers.Dense(action_size, activation='linear', kernel_initializer=init))
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 25)                1625      
                                                                 
 dense_1 (Dense)             (None, 25)                650       
                                                                 
 dense_2 (Dense)             (None, 4)                 104       
                                                                 
Total params: 2,379
Trainable params: 2,379
Non-trainable params: 0
_________________________________________________________________


 Czas nauczyć agenta poruszania się po środowisku *FrozenLake*, jako stan przyjmij wektor o liczbie elementów równej liczbie możliwych stanów, z wartością 1 ustawioną w komórce o indeksie równym aktualnemu stanowi, pozostałe elementy mają być wypełnione zerami:
* 1 pkt < 35 epok,
* 0.5 pkt < 60 epok,
* 0.25 pkt - w pozostałych przypadkach.

In [ ]:
agent = DQNAgent(action_size, learning_rate, model, frozenLake("8x8"))

agent.epsilon = 1
batch_size = 100
EPISODES = 100
for e in range(EPISODES):

    summary = []
    pbar = tqdm(range(30))
    for _ in pbar:
        total_reward = 0
        env_state = env.reset()

        state = np.zeros((1, env.get_number_of_states()))
        state[0, env._current_state] = 1

        while True:
            action = agent.get_action(state)
            next_state_env, reward, done, _ = env.step(action)
            total_reward += reward
            next_state = np.zeros((1, env.get_number_of_states()))
            next_state[0, env._current_state] = 1

            #add to experience memory
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                break

        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
            agent.update_epsilon_value()

        summary.append(total_reward)
        pbar.set_description(f'training epoch')
    print("epoch #{}\tmean reward = {:.3f}\tepsilon = {:.3f}".format(e, np.mean(summary), agent.epsilon))
    if np.mean(summary) > 0.9:
        print ("You Win!")
        break

training epoch: 100%|██████████| 30/30 [08:19<00:00, 16.66s/it]


epoch #0	mean reward = 0.033	epsilon = 0.740


training epoch: 100%|██████████| 30/30 [08:20<00:00, 16.67s/it]


epoch #1	mean reward = 0.000	epsilon = 0.547


training epoch: 100%|██████████| 30/30 [08:23<00:00, 16.77s/it]


...

training epoch: 100%|██████████| 30/30 [08:24<00:00, 16.83s/it]


epoch #3	mean reward = 0.000	epsilon = 0.299


training epoch: 100%|██████████| 30/30 [08:27<00:00, 16.93s/it]


epoch #4	mean reward = 0.000	epsilon = 0.221


  0%|          | 0/30 [00:00<?, ?it/s]

Czas przygotować model sieci, która będzie się uczyła poruszania po środowisku *FrozenLakeExtended*, tym razem stan nie jest określany poprzez pojedynczą liczbę, a przez 3 tablice:
* pierwsza zawierająca informacje o celu,
* druga zawierająca informacje o dziurach,
* trzecia zawierająca informację o położeniu gracza.

In [ ]:
env = frozenLakeExtended("4x4")

state_size = env.get_number_of_states()
action_size = len(env.get_possible_actions(None))
learning_rate = 0.001

model =         
        #
        # INSERT CODE HERE to build network
        #

 Czas nauczyć agenta poruszania się po środowisku *FrozenLakeExtended*, jako stan przyjmij wektor składający się ze wszystkich trzech tablic (2 pkt.):

In [ ]:
agent = DQNAgent(action_size, learning_rate, model)

agent.epsilon = 0.75

done = False
batch_size = 64
EPISODES = 2000
counter = 0
for e in range(EPISODES):
    summary = []
    for _ in range(100):
        total_reward = 0
        env_state = env.reset()
    
        #
        # INSERT CODE HERE to prepare appropriate format of the state for network
        #
        
        for time in range(1000):
            action = agent.get_action(state)
            next_state_env, reward, done, _ = env.step(action)
            total_reward += reward

            #
            # INSERT CODE HERE to prepare appropriate format of the next state for network
            #

            #add to experience memory
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                break

        #
        # INSERT CODE HERE to train network if in the memory is more samples then size of the batch
        #
        
        summary.append(total_reward)
    if np.mean(summary) > 0.9:
        print ("You Win!")
        break
    print("epoch #{}\tmean reward = {:.3f}\tepsilon = {:.3f}".format(e, np.mean(summary), agent.epsilon))

Czas przygotować model sieci, która będzie się uczyła działania w środowisku [*CartPool*](https://gym.openai.com/envs/CartPole-v0/):

In [ ]:
env = gym.make("CartPole-v0").env
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
learning_rate = 0.001

model =         
        #
        # INSERT CODE HERE to build network
        #

Czas nauczyć agenta gry w środowisku *CartPool*:
* 1 pkt < 10 epok,
* 0.5 pkt < 20 epok,
* 0.25 pkt - w pozostałych przypadkach.

In [ ]:
agent = DQNAgent(action_size, learning_rate, model)

agent.epsilon = 0.75

done = False
batch_size = 64
EPISODES = 1000
counter = 0
for e in range(EPISODES):
    summary = []
    for _ in range(100):
        total_reward = 0
        env_state = env.reset()
    
        #
        # INSERT CODE HERE to prepare appropriate format of the state for network
        #
        
        for time in range(300):
            action = agent.get_action(state)
            next_state_env, reward, done, _ = env.step(action)
            total_reward += reward

            #
            # INSERT CODE HERE to prepare appropriate format of the next state for network
            #

            #add to experience memory
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                break

        #
        # INSERT CODE HERE to train network if in the memory is more samples then size of the batch
        #
        
        summary.append(total_reward)
    if np.mean(summary) > 195:
        print ("You Win!")
        break
    print("epoch #{}\tmean reward = {:.3f}\tepsilon = {:.3f}".format(e, np.mean(summary), agent.epsilon))